# Tugas Akhir
"Klasfikasi Perintah Bahasa Natural Menggunakan *Global Vectors for Word Representations* (GloVe), *Convolutional Neural Networks*, dan Teknik *Transfer Learning* Untuk Aplikasi *Chatbots*"<br>

Irfan Hanif<br>
5114100177

### Menyiapkan Source Dataset
Source Dataset akan digunakan untuk melatih CNN sebelum dilakukan proses Transfer Learning

## Penjelasan

Langkah pertama yang dilakukan adalah meng-import modul pandas dan numpy serta mengambil data lapor.csv pada folder di data_path

In [43]:
import pandas as pd
import numpy as np

In [44]:
data_path = "data/data_target/"
filename = "questions.csv"

Membaca file lapor.csv dengan memanfaatkan modul Pandas dengan menggunakan fungsi .head(), maka data yang ditampilkan merupakan 5 data teratas 

In [45]:
train_df = pd.read_csv(data_path + filename, delimiter=',')
train_df.head()

,pertanyaan,Kategori
0,bagaimana kisah tentang mukasyafah nya sayyidi...,KISAH SAYYIDINA UMAR VS MALAIKAT MUNKAR-NAKIR
1,ceritakan kisah sayyidina umar vs munkar nakir,KISAH SAYYIDINA UMAR VS MALAIKAT MUNKAR-NAKIR
2,cerita sayyidina umar vs munkar nakir termaktu...,KISAH SAYYIDINA UMAR VS MALAIKAT MUNKAR-NAKIR
3,kenapa wudhu menggunakan air?,ALASAN BERSUCI DENGAN AIR
4,apakah bisa wudhu tidak menggunakan air?,ALASAN BERSUCI DENGAN AIR


menampilkan jumlah masing-masing kategori

In [46]:
kategori_total_list = train_df["Kategori"].value_counts()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print (kategori_total_list)

ALASAN BERSUCI DENGAN AIR                                                  13
TAFSIR MIMPI MASJID ROBOH                                                   5
MAHAR NIKAH BERASAL DARI ORANG LAIN                                         5
TAJWID : MAD LAZIM HARFI MUSYABBA?                                          5
HUKUM BERQURBAN DENGAN AYAM JAGO                                            4
HUKUM LAKI-LAKI MEMAKAI MINYAK ZA?FARON                                     4
HUKUM BERBUAT DOSA BAGI ANAK YANG BELUM BALIGH II                           4
JIKA MAKMUM KETINGGALAN BACA FATIHAH KARENA IMAM SHOLAT TERLALU CEPAT       4
hukum sholat qobliyah shubuh setelah shalat shubuh                          4
SHOLAT : HUKUM BACA BASMALAH SAAT AKAN SHOLAT                               4
HUKUM MELUDAH KE ARAH KIBLAT                                                4
HUKUM JUAL BELI KUCING                                                      4
dalil posisi duduk saat zikir ba'da shalat maktubah             

menggabungkan kategori yang memiliki jumlah berita sedikit dengan list kategori lain yang lebih umum dan memiliki jumlah yang sudah banyak.

In [47]:
kategori = train_df["Kategori"].values

Menampilkan kembali hasil kategori setelah di gabungkan

In [48]:
kategori_total_list = train_df["Kategori"].value_counts()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print (kategori_total_list)

ALASAN BERSUCI DENGAN AIR                                                  13
TAFSIR MIMPI MASJID ROBOH                                                   5
MAHAR NIKAH BERASAL DARI ORANG LAIN                                         5
TAJWID : MAD LAZIM HARFI MUSYABBA?                                          5
HUKUM BERQURBAN DENGAN AYAM JAGO                                            4
HUKUM LAKI-LAKI MEMAKAI MINYAK ZA?FARON                                     4
HUKUM BERBUAT DOSA BAGI ANAK YANG BELUM BALIGH II                           4
JIKA MAKMUM KETINGGALAN BACA FATIHAH KARENA IMAM SHOLAT TERLALU CEPAT       4
hukum sholat qobliyah shubuh setelah shalat shubuh                          4
SHOLAT : HUKUM BACA BASMALAH SAAT AKAN SHOLAT                               4
HUKUM MELUDAH KE ARAH KIBLAT                                                4
HUKUM JUAL BELI KUCING                                                      4
dalil posisi duduk saat zikir ba'da shalat maktubah             

Menampilkan isi laporan

In [49]:
pertanyaan = train_df["pertanyaan"].values
pertanyaan

array(['bagaimana kisah tentang mukasyafah nya sayyidina ali r.a tentang sayyidina umar vs mungkar nakir',
       'ceritakan kisah sayyidina umar vs munkar nakir',
       'cerita sayyidina umar vs munkar nakir termaktub dari kitab apa?',
       'kenapa wudhu menggunakan air?',
       'apakah bisa wudhu tidak menggunakan air?',
       'kalau tidak ada air bagaimana bisa berwuduhu?', 'wudhu tanpa air',
       'mengapa wuduh harus menggunakan air?', 'haruskah wudhu pake air?',
       "kenapa sholat maghrib 3 rokaat. Ashar 4 Zhuhur 4 Subuh 2 roka'at. Kok tidak disamakan saja",
       'mengapa jumlah rakaat sholat beragam',
       'mengapa jumlah rakaat sholat berbeda-beda',
       'mengapa jumlah rakaat sholat tidak sama',
       'kenapa sholat maghrib 3 rakaat',
       'mengapa sholat dzuhur ashar dan isya 4 rakaat',
       'kenapa sholat subuh 2 rakaat',
       'bolehkah mahar nikah berasal dari orang lain?',
       'calon suami mendapat sponsor untuk dijadikan mahar nikah seperti pada a

Mendapatkan word embedding dari database glove

In [50]:
def getWordEmbedding(word, cursor):
#     word = word.replace("'", "''")
    sql = """select vec from term where term like %s"""
    cursor.execute(sql, (str(word),))
    data = cursor.fetchall()
    if len(data) > 0:
        decoded_vec = json.JSONDecoder().decode(data[0][0])
        vec = np.asarray(decoded_vec, dtype=np.float32)
        return True, vec
    else:
        return False, data
    
def myTokenizer(content, lower=True):
    raw = content.split(' ')
    remover = re.compile("[^a-zA-Z-]")
    
    token = []
    
    for i in raw:
        term = remover.sub('', i)
        if lower == True:
            term = term.lower()
        token.append(term)
#     tokenized = filter(None, token)
    
#     return tokenized
    return token

def sentenceToVec(string):

    if type(string) is not str:
        return np.zeros((300,), dtype=float)
   
    string = string.replace('\n', '')
    string = np.array(myTokenizer(string))
#     print(string)
    feature = None
    begin = True
    for word in string:
        stat, vec = getWordEmbedding(word, cursor)
        if not stat:
            continue
        if begin:
            begin = False
            feature = vec
        else:
            feature += vec
#             feature = np.concatenate([feature, vec])
#     print('running') 
    if feature is not None:
        feature = feature/np.linalg.norm(feature)
    else:
        feature = np.zeros((300,), dtype=float)
    
    return feature

import re
import json
import unicodedata
import mysql.connector
from tqdm import tqdm

db = mysql.connector.connect(user="root", password='', database="glove")
cursor = db.cursor(buffered=True)

pbar = tqdm(total=len(pertanyaan))
ftr = []
for i in pertanyaan:
    try:
        ftr.append(sentenceToVec(i))
    except Exception as err:
        print (err)
        print (i)
        break
    pbar.update(1)
pbar.close()
ftr = np.array(ftr)

100%|████████████████████████████████████████████████████████████████████████████████| 222/222 [00:14<00:00, 13.32it/s]


In [51]:
nan_idx = []
for i in range(len(kategori)):
    if kategori[i] is np.nan:
        nan_idx.append(i)
kategori_c = np.delete(kategori, nan_idx)
ftr_c = np.delete(ftr, nan_idx, 0)

In [52]:
cls = []
kategori_u = pd.unique(kategori_c)
kategori_u = kategori_u.tolist()
for i in range(len(kategori_c)):
    one_hot = np.zeros((len(kategori_u),), dtype=int)
    idx = kategori_u.index(kategori_c[i])
    one_hot[idx] = 1
    cls.append(one_hot)
cls = np.array(cls)

print ("Mengecek array size dari fitur dan kelas...")
print ("size of ftr " + str(ftr_c.shape))
print ("size of cls " + str(cls.shape))

Mengecek array size dari fitur dan kelas...
size of ftr (222, 300)
size of cls (222, 73)


In [53]:
np.save(data_path + "ftr-data-source.npy", ftr_c)
np.save(data_path + "cls-data-source.npy", cls)

## SMOTE
Source Dataset ternyata tidak imbang antara satu kelas dan kelas lainnya. Maka, digunakan algoritma **SMOTE** untuk membuat jumlah data dan setiap kelasnya imbang (*balanced*).

In [54]:
ftr = np.load(data_path + "ftr-data-source.npy")
cls = np.load(data_path + "cls-data-source.npy")

In [55]:
indexed_cls = list(map(lambda x: np.argmax(x), cls))
print(pd.unique(indexed_cls))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72]


In [56]:
def valuesCount(arr):
    cls_dict = {}
    for i in arr:
        if i in cls_dict:
            cls_dict[i] += 1
        else:
            cls_dict[i] = 1
    return cls_dict

In [57]:
valuesCount(indexed_cls)

{0: 3,
 1: 13,
 2: 5,
 3: 3,
 4: 5,
 5: 3,
 6: 3,
 7: 3,
 8: 3,
 9: 4,
 10: 5,
 11: 3,
 12: 2,
 13: 4,
 14: 4,
 15: 3,
 16: 3,
 17: 4,
 18: 4,
 19: 3,
 20: 3,
 21: 3,
 22: 3,
 23: 3,
 24: 2,
 25: 2,
 26: 4,
 27: 1,
 28: 2,
 29: 3,
 30: 2,
 31: 2,
 32: 3,
 33: 2,
 34: 3,
 35: 3,
 36: 4,
 37: 2,
 38: 3,
 39: 3,
 40: 3,
 41: 1,
 42: 3,
 43: 3,
 44: 3,
 45: 4,
 46: 3,
 47: 2,
 48: 3,
 49: 3,
 50: 3,
 51: 1,
 52: 2,
 53: 3,
 54: 3,
 55: 3,
 56: 3,
 57: 3,
 58: 3,
 59: 3,
 60: 3,
 61: 3,
 62: 3,
 63: 3,
 64: 3,
 65: 1,
 66: 3,
 67: 3,
 68: 2,
 69: 1,
 70: 3,
 71: 3,
 72: 3}

In [58]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
ftr_res, cls_res = sm.fit_sample(ftr, indexed_cls)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 3, n_neighbors = 6

In [59]:
valuesCount(cls_res)

NameError: name 'cls_res' is not defined

In [60]:
def convToOneHot(arr):
    total_cls = len(pd.unique(arr))
    cls = []
    for i in arr:
        temp_vec = np.zeros((total_cls,), dtype=int)
        temp_vec[i] = 1
        cls.append(temp_vec)
    cls = np.array(cls)
    return cls

In [61]:
cls_fin = convToOneHot(cls_res)
print ("Panjang array kelas seletah convToOneHot: " + str(len(cls_fin[0]))

SyntaxError: unexpected EOF while parsing (<ipython-input-61-4a6b73fc7202>, line 2)

In [39]:
np.save(data_path + "ftr_data_source_balanced.npy", ftr_res)
np.save(data_path + "cls_data_source_balanced.npy", cls_fin)

NameError: name 'ftr_res' is not defined

## Menyiapkan Target Dataset
Menyiapkan Target Dataset, mengubah bentuknya agar bisa menjadi masukan untuk CNN.

In [40]:
import re
import json
import unicodedata
import mysql.connector
import numpy as np
import pandas as pd

In [41]:
data_target_path = "data/data_target/"
filename_data_target = "data_target_schematics.csv"

In [42]:
def arrangeInputMatrix(data, labels):
    ftr_list = []
    cls_list = []
    
    unique_labels = pd.unique(labels)
    total_labels = len(unique_labels)
    
    for i in range(len(data)):
        cls = np.zeros((total_labels,), dtype=int)
        cls_idx = np.where(unique_labels == labels[i])[0][0]
        cls[cls_idx] = 1
        
        string = data[i].replace('\n', '')
        string = np.array(myTokenizer(string))

        begin = True
        for word in string:
            stat, vec = getWordEmbedding(word, cursor)
            if not stat:
                continue
            if begin:
                begin = False
                feature = vec
            else:
                feature += vec
                # feature = np.concatenate([feature, vec])

        feature = feature/np.linalg.norm(feature)
        ftr_list.append(feature)
        cls_list.append(cls)
        
    return np.array(ftr_list), np.array(cls_list)

In [2]:
print ("convert into numpy array")
df = pd.read_csv(data_target_path + filename_data_target, sep=',', names=["documents", "labels"])
documents = df["documents"].values
labels = df["labels"].values

convert into numpy array


NameError: name 'pd' is not defined

In [1]:
ftr, cls = arrangeInputMatrix(documents, labels)

NameError: name 'arrangeInputMatrix' is not defined

In [23]:
print "checking len of array"
print len(documents) == len(ftr)
print len(documents) == len(cls)

print
print "checking size of arrays"
print "size of ftr: " + str(ftr.shape)
print "size pf cls: " + str(cls.shape)

checking len of array
True
True

checking size of arrays
size of ftr: (462L, 300L)
size pf cls: (462L, 22L)


In [24]:
np.save(data_target_path + "ftr_data_target.npy", ftr)
np.save(data_target_path + "cls_data_target.npy", cls)

## Splitting Dataset
Memisahkan Source Dataset dan Target Dataset menjadi Train Set dan Test Tes

## Splitting Target Dataset

In [35]:
ftr_data_target = np.load(data_path + "ftr-data-source.npy")
cls_data_target = np.load(data_path + "cls-data-source.npy")

In [36]:
data_target_path_splitted = "data/splitted/data_target/"

In [37]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.5, random_state=0)
for train_index, test_index in sss.split(ftr_data_target, cls_data_target):
    X, X_test = ftr_data_target[train_index], ftr_data_target[test_index]
    Y, Y_test = cls_data_target[train_index], cls_data_target[test_index]
    break
# X = np.expand_dims(X, axis=2)
# X_test = np.expand_dims(X_test, axis=2)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [29]:
np.save(data_target_path_splitted + "X.npy", X)
np.save(data_target_path_splitted + "Y.npy", Y)
np.save(data_target_path_splitted + "X_test.npy", X_test)
np.save(data_target_path_splitted + "Y_test.npy", Y_test)

## Splitting Source Dataset

In [28]:
data_source_path = "data/data_source/"

ftr_data_source = np.load(data_source_path + "ftr_data_source_balanced.npy")
cls_data_source = np.load(data_source_path + "cls_data_source_balanced.npy")

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_source/ftr_data_source.npy'

In [31]:
data_source_path_splitted = "data/splitted/data_source/"

In [33]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.1, random_state=0)
for train_index, test_index in sss.split(ftr_data_source, cls_data_source):
    X, X_test = ftr_data_source[train_index], ftr_data_source[test_index]
    Y, Y_test = cls_data_source[train_index], cls_data_source[test_index]
    break
# X = np.expand_dims(X, axis=2)
# X_test = np.expand_dims(X_test, axis=2)

In [34]:
np.save(data_source_path_splitted + "X.npy", X)
np.save(data_source_path_splitted + "Y.npy", Y)
np.save(data_source_path_splitted + "X_test.npy", X_test)
np.save(data_source_path_splitted + "Y_test.npy", Y_test)